In [1]:
'''
https://towardsdatascience.com/machine-learning-word-embedding-sentiment-classification-using-keras-b83c28087456
http://ruder.io/deep-learning-nlp-best-practices/
https://skymind.ai/wiki/word2vec
https://medium.com/@ppasumarthi_69210/word-embeddings-in-keras-be6bb3092831
https://towardsdatascience.com/neural-network-embeddings-explained-4d028e6f0526
https://medium.com/@dcameronsteinke/tf-idf-vs-word-embedding-a-comparison-and-code-tutorial-5ba341379ab0
https://medium.com/@b.terryjack/nlp-everything-about-word-embeddings-9ea21f51ccfe
https://towardsdatascience.com/word-embeddings-exploration-explanation-and-exploitation-with-code-in-python-5dac99d5d795
'''

'\nhttps://towardsdatascience.com/machine-learning-word-embedding-sentiment-classification-using-keras-b83c28087456\nhttp://ruder.io/deep-learning-nlp-best-practices/\nhttps://skymind.ai/wiki/word2vec\nhttps://medium.com/@ppasumarthi_69210/word-embeddings-in-keras-be6bb3092831\nhttps://towardsdatascience.com/neural-network-embeddings-explained-4d028e6f0526\nhttps://medium.com/@dcameronsteinke/tf-idf-vs-word-embedding-a-comparison-and-code-tutorial-5ba341379ab0\nhttps://medium.com/@b.terryjack/nlp-everything-about-word-embeddings-9ea21f51ccfe\nhttps://towardsdatascience.com/word-embeddings-exploration-explanation-and-exploitation-with-code-in-python-5dac99d5d795\n'

In [2]:
from __future__ import print_function

import csv
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('rslp')

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/konpyro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/konpyro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/konpyro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package rslp to /home/konpyro/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [3]:
# X: input, Y:output
X = []
Y = []




with open('MoodyLyricsFull4Q.csv') as file:
    reader = csv.reader(file, delimiter = ',')
    count = 0
    for row in reader:
        if count > 0:
            X.append(row[4])
            Y.append(row[3])
        count = count + 1

In [4]:
#Y = np.concatenate(y_train, y_test)
y_t = np.zeros((len(Y), 4))
for i  in range(len(Y)):
    if Y[i] == 'happy':
        y_t[i][:] = [1, 0, 0, 0]
    elif Y[i] == 'angry':
        y_t[i][:] = [0, 1, 0, 0]
    elif Y[i] == 'sad':
        y_t[i][:] = [0, 0, 1, 0]
    elif Y[i] == 'relaxed':
        y_t[i][:] = [0, 0, 0, 1]
Y = y_t

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [6]:
t = Tokenizer()
total_lyrics = x_train + x_test
t.fit_on_texts(total_lyrics)

max_length = max([len(s.split()) for s in total_lyrics])
vocab_size = len(t.word_index) + 1
EMBEDDING_DIM = 100

x_train_tokens = t.texts_to_sequences(x_train)
x_test_tokens = t.texts_to_sequences(x_test)

x_train_pad = pad_sequences(x_train_tokens, padding='post')
x_test_pad = pad_sequences(x_test_tokens, padding='post')

print(x_train_tokens[1])
print(x_train_pad[1])

[2, 101, 66, 7, 505, 2, 101, 2133, 9, 1965, 22, 2, 60, 2, 59, 1, 5455, 11, 5456, 38, 2, 298, 9, 419, 25, 43, 1, 46, 10, 1, 69, 11, 1, 93, 11, 2810, 60, 41, 2, 91, 9, 792, 5, 4775, 12, 1179, 4, 38, 3, 107, 11, 8, 9, 906, 199, 5, 3487, 3, 56, 162, 344, 5, 2, 89, 25, 23, 366, 30, 3, 166, 164, 377, 7, 26, 12, 1356, 370, 19, 201, 7, 252, 148, 7, 44, 66, 7, 269, 39, 145, 35, 60, 8, 3836, 21, 127, 221, 164, 21, 703, 3233, 10, 12, 247, 282, 2, 1130, 1, 1819, 3, 52, 724, 7, 2, 1429, 1, 414, 1820, 60, 2, 66, 8, 153, 153, 153, 613, 14, 1, 11148, 4198, 4, 8002, 2, 11149, 9, 614, 115, 2, 748, 9, 604, 4, 38, 3, 107, 11, 8, 9, 906, 199, 5, 3487, 3, 56, 162, 344, 5, 2, 89, 25, 23, 366, 30, 3, 166, 164, 377, 7, 26, 12, 1356, 370, 19, 201, 7, 252, 148, 7, 44, 66, 7, 269, 39, 145, 35, 60, 8, 3836, 21, 127, 221, 164, 21, 703, 3233, 10, 12, 247, 282]
[    2   101    66     7   505     2   101  2133     9  1965    22     2
    60     2    59     1  5455    11  5456    38     2   298     9   419
    25    43

In [7]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

lyric_lines = list()
lines = X

for line in lines:
    tokens = word_tokenize(line)

    tokens = [w.lower() for w in tokens]

    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]

    words = [word for word in stripped if word.isalpha()]

    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    lyric_lines.append(words)

In [8]:
import gensim

model = gensim.models.Word2Vec(sentences=lyric_lines, size=100, window=5, workers=4, min_count=1)

words = list(model.wv.vocab)
print("Vocabulary size: %d" %len(words))

Vocabulary size: 20634


In [9]:
model.wv.most_similar('love')

[('mine', 0.9754360914230347),
 ('privacy', 0.971505880355835),
 ('kiss', 0.9699615240097046),
 ('true', 0.9668830633163452),
 ('feels', 0.965377151966095),
 ('darling', 0.9638769030570984),
 ('sure', 0.9632102251052856),
 ('thing', 0.9623692631721497),
 ('affection', 0.9617385864257812),
 ('please', 0.9617030024528503)]

In [10]:
''''
filename = 'lyric_embedding_word2vec'
model.wv.save(filename)

import os

embeddings_index = {}
f = open(os.path.join('', 'lyric_embedding_word2vec.txt'), encoding='latin-1')
for line in f:
  print(line)
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:])
  embeddings_index[word] = coefs
f.close()
'''

"'\nfilename = 'lyric_embedding_word2vec'\nmodel.wv.save(filename)\n\nimport os\n\nembeddings_index = {}\nf = open(os.path.join('', 'lyric_embedding_word2vec.txt'), encoding='latin-1')\nfor line in f:\n  print(line)\n  values = line.split()\n  word = values[0]\n  coefs = np.asarray(values[1:])\n  embeddings_index[word] = coefs\nf.close()\n"

In [11]:
my_dict = dict({})
for idx, key in enumerate(model.wv.vocab):
    my_dict[key] = model.wv[key]


In [12]:
# %store my_dict > dictionary.txt


In [13]:
embeddings_index = my_dict

In [14]:
t = Tokenizer()
t.fit_on_texts(lyric_lines)
sequences = t.texts_to_sequences(lyric_lines)

word_index = t.word_index
print('Found %s unique tokens.' %len(word_index))

review_pad = pad_sequences(sequences, maxlen=max_length)
sentiment = np.asarray(Y)
print('Shape of lyric tensor:', review_pad.shape)
print('Shape of sentiment tensor;', sentiment.shape)


Found 20634 unique tokens.
Shape of lyric tensor: (2000, 1015)
Shape of sentiment tensor; (2000, 4)


In [15]:
num_words = len(word_index) + 1
embeddings_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector
        
print(num_words)

20635


In [16]:
from keras.initializers import Constant

print('Biuld model...')
model = Sequential()
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embeddings_matrix),
                            input_length=max_length,
                            trainable=False)
model.add(embedding_layer)
#model.add(LSTM(32, dropout=0.2, return_sequences=True, recurrent_dropout=0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Biuld model...


In [17]:
VALIDATION_SPLIT = 0.33

indices = np.arange(review_pad.shape[0])
np.random.shuffle(indices)
review_pad = review_pad[indices]
sentiment = sentiment[indices]
num_validation_samples = int(VALIDATION_SPLIT * review_pad.shape[0])

x_train_pad = review_pad[:-num_validation_samples]
y_train = sentiment[:-num_validation_samples]
x_test_pad = review_pad[-num_validation_samples:]
y_test = sentiment[-num_validation_samples:]

In [18]:
print('Shape of x_train_pad tensor:', x_train_pad.shape)
print('Shape of y_train tensor:', y_train.shape)
print('Shape of x_test_pad tensor:', x_test_pad.shape)
print('Shape of y_test tensor:', y_test.shape)


Shape of x_train_pad tensor: (1340, 1015)
Shape of y_train tensor: (1340, 4)
Shape of x_test_pad tensor: (660, 1015)
Shape of y_test tensor: (660, 4)


In [19]:
print('Train model...')

model.fit(x_train_pad, y_train, epochs=10, validation_data=(x_test_pad, y_test), verbose=2)

Train model...
Train on 1340 samples, validate on 660 samples
Epoch 1/10
 - 34s - loss: 1.3806 - accuracy: 0.2963 - val_loss: 1.3456 - val_accuracy: 0.3379
Epoch 2/10
 - 38s - loss: 1.3650 - accuracy: 0.3119 - val_loss: 1.3327 - val_accuracy: 0.3485
Epoch 3/10
 - 34s - loss: 1.3432 - accuracy: 0.3455 - val_loss: 1.3318 - val_accuracy: 0.3561
Epoch 4/10
 - 39s - loss: 1.3353 - accuracy: 0.3530 - val_loss: 1.3313 - val_accuracy: 0.3606
Epoch 5/10
 - 37s - loss: 1.3208 - accuracy: 0.3634 - val_loss: 1.3324 - val_accuracy: 0.3682
Epoch 6/10
 - 42s - loss: 1.3104 - accuracy: 0.3784 - val_loss: 1.3373 - val_accuracy: 0.3591
Epoch 7/10
 - 39s - loss: 1.3062 - accuracy: 0.3657 - val_loss: 1.3169 - val_accuracy: 0.3697
Epoch 8/10
 - 34s - loss: 1.3013 - accuracy: 0.3866 - val_loss: 1.3172 - val_accuracy: 0.3727
Epoch 9/10
 - 34s - loss: 1.2914 - accuracy: 0.4007 - val_loss: 1.3049 - val_accuracy: 0.3561
Epoch 10/10
 - 35s - loss: 1.2916 - accuracy: 0.3873 - val_loss: 1.3146 - val_accuracy: 0.36

In [20]:
loss, acc = model.evaluate(x_test_pad, y_test)
print('Test loss:', loss)
print('Test accuracy:', acc)


660/660 [==============================] - 7s 11ms/step
Test loss: 1.3145509553678107
Test accuracy: 0.36666667461395264


LSTM(64),
split: 0.33,
epochs: 5,
dropout=r_dropout = 0.2,
-> **loss: 1.31, acc: 0.36**


